In [49]:
%pip install lxml


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
from lxml import etree
from pathlib import Path

Tf-Idf for the three homer documents in the tlg0012 folder

In [51]:
#opening files
files = Path("tlg0012").glob("./**/*perseus-eng*.xml") #** is search all the subdir, then search for perseus-eng with anything after it
#it will look at each of the files and match each of the files w perseus-eng

# for f in files:
#     print(f)

In [52]:
TEI_NS = "{http://www.tei-c.org/ns/1.0}"
XML_NS = "{http://www.w3.org/XML/1998/namespace}"

NAMESPACES = {
    "tei": TEI_NS,
    "xml": XML_NS
}

In [53]:
# iterate thru each of the files and parse using xml
for file in files:
    print(file)
    tree = etree.parse(file)
    text = tree.xpath(f"//tei:div[@subtype='card']//text()", namespaces=NAMESPACES)
    
    cleaned_text = []
    for t in text:
        if t.strip() != "":
            cleaned_text.append(t.strip())

    if len(cleaned_text) > 0:
        with open(str(file).split("/")[-1].replace(".xml", ".txt"), "w+") as f:
            f.write("\n".join(cleaned_text))

tlg0012/tlg001/tlg0012.tlg001.perseus-eng3.xml
tlg0012/tlg001/tlg0012.tlg001.perseus-eng4.xml
tlg0012/tlg003/tlg0012.tlg003.perseus-eng1.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng4.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng3.xml


## End of TF-IDF Recap

In [39]:
for file in files:
    print(file)
    tree = etree.parse(file)
    for p in tree.iterfind(f"//{TEI_NS}text//{TEI_NS}p"): #saying find every p element (in the tei namespace) in the text and get any text in that element(p)
        print(p.text)


 however now we only get text within a p element; so whenever there's text in a diff element (ie "note") it produces whitespace

In [40]:
from collections import Counter

text_files = list(Path(".").glob("tlg0012.tlg00*.perseus-eng*.txt"))
counts = {}

for t in text_files:
    name = str(t) # key in counts

    with open(t) as f:
        text = f.read().lower().split() #reading, then lowercasing, and finally splitting on yt-space
        counts[name] = Counter(text)


In [41]:
# to get idf's
term = 'ulysses'
df_ulysses = 0
for _, els in counts.items():
    if term in els:
        df_ulysses += 1

df_ulysses
#[1 if term in counts.ke] # doc freq is 1 if the term is in 

0

## TF-IDF in Action

In [55]:
%pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
import nltk

# download the files needed for tokenization
# the punkt tokenizer should be installed already,
# but let's download it just in case
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

#### Tokenize text
With the tokenizer downloaded, we can now read in and tokenize each text.

In [57]:
# Initialize the tokenizer
from nltk.tokenize import word_tokenize

# Initialize an empty dictionary to store the tokenized texts
tokenized_texts = {}

# Get a Path.glob() iterator for the .txt files that you've created in this directory.
# Can you figure out what the new `[1-4]` segment is doing?
text_files = Path(".").glob("tlg0012.tlg00*.perseus-eng[1-4].txt")

# Iterate through the text files, reading and tokenizing them one by one,
# then storing the list of tokens in our `tokenized_texts` dictionary —
# so we'll be getting a dictionary of lists.
for file in text_files:
    name = str(file)

    with open(file) as f:
        # Notice we're lowercasing the text. You don't *have*
        # to do this, but it helps eliminate some noise for
        # our purposes.
        text = f.read().lower()
        tokens = word_tokenize(text)

        # Let's just print the length of the tokens list to make
        # sure we're getting sane results. We'll use string interpolation
        # to identify which text we're working with.
        print(f"There are {len(tokens)} tokens in {name}.")

        # Store each file's `tokens` list in the `tokenized_texts`
        # dictionary, using the filename as the key.
        tokenized_texts[name] = tokens

There are 200630 tokens in tlg0012.tlg001.perseus-eng3.txt.
There are 135463 tokens in tlg0012.tlg002.perseus-eng4.txt.
There are 175611 tokens in tlg0012.tlg001.perseus-eng4.txt.
There are 152631 tokens in tlg0012.tlg002.perseus-eng3.txt.


#### Count the Tokens
Now, we could count these tokens by hand, but why do that when Python gives us the Counter object?

In [58]:
from collections import Counter

# Using our `tokenized_texts` dictionary, we'll iterate
# through each key-value pair — remember, the keys are
# filenames and the values are lists of tokens.
# We'll get a count of the tokens by passing the list to
# `Counter`, then we'll change the value for that key to
# a dictionary with its own keys, `tokens` and `counts`.

for filename, tokens in tokenized_texts.items():
    counts = Counter(tokens)

    tokenized_texts[filename] = {"tokens": tokens, "counts": counts}

In [59]:
tokenized_texts["tlg0012.tlg001.perseus-eng3.txt"]["counts"]["odysseus"]

128

#### Calculate the document frequency for a given term
Let’s compare occurrences of the strings "odysseus" and "achilles" — we probably expect the former to “matter” more for the Odysseus, and the latter for the Iliad. Let’s see if that’s the case.

In [60]:
df_achilles = 0
df_odysseus = 0

# Calculate the DF for "odysseus" and "achilles".
# We iterate through the dictionary, and then simply
# count the number of files in which we find each term.
# For these two terms, we should probably expect DFs of 4.
for filename, values in tokenized_texts.items():
    if "odysseus" in values['counts']:
        df_odysseus += 1
    
    if "achilles" in values["counts"]:
        df_achilles += 1

# Now we'll import the log function to calculate the IDF for each term.
from math import log10

n_docs = len(tokenized_texts.keys())

idf_achilles = log10(n_docs / df_achilles)
idf_odysseus = log10(n_docs / df_odysseus)

In [61]:
# Now let's calculate the TF-IDF "score" for each term in each document.

# Once again, iterate through the dictionary.
for filename, values in tokenized_texts.items():
    # Get the total number of terms in each file — we'll
    # use this to calculate the relative frequency as our
    # TF.
    total_terms = len(values['tokens'])
    print(total_terms)
    # Get the TF for each term in this file.
    tf_achilles = values['counts']['achilles'] / total_terms
    tf_odysseus = values['counts']['odysseus'] / total_terms

    # Remember, the simplest version of TF-IDF is just
    # TF * 1/DF
    tf_idf_achilles = tf_achilles * idf_achilles
    tf_idf_odysseus = tf_odysseus * idf_odysseus

    # Now we can report on the statistics for this file
    print(f"""In {filename}:
TF of achilles: {tf_achilles}
TF of odysseus: {tf_odysseus}
TF-IDF of achilles: {tf_idf_achilles}
TF-IDF of odysseus: {tf_idf_odysseus}
""")

200630
In tlg0012.tlg001.perseus-eng3.txt:
TF of achilles: 0.002043562777251657
TF of odysseus: 0.000637990330459054
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

135463
In tlg0012.tlg002.perseus-eng4.txt:
TF of achilles: 0.0001254955227626732
TF of odysseus: 0.0042816119530794386
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

175611
In tlg0012.tlg001.perseus-eng4.txt:
TF of achilles: 0.002403038534032606
TF of odysseus: 0.0007061061095261686
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

152631
In tlg0012.tlg002.perseus-eng3.txt:
TF of achilles: 0.0001048279838302835
TF of odysseus: 0.0041603606082643765
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0



### Set Exercise


In [62]:
my_list = [1, 1, 2, 3, 3]

set(my_list)

{1, 2, 3}

In [63]:
non_universal_terms = {}

for filename, values in tokenized_texts.items():
    my_set = set(values['counts'].keys())

    for other_file, other_values in tokenized_texts.items():
        # make sure we don't compare the file
        # to itself, otherwise the difference
        # will be the empty set
        if other_file != filename:
            my_set -= set(other_values['counts'].keys())
    
    # now push the remaining set of terms to the dictionary
    non_universal_terms[filename] = my_set

# log `non_universal_terms` as a sanity check
non_universal_terms

{'tlg0012.tlg001.perseus-eng3.txt': {'ungirt',
  'cudgels',
  '249.1',
  'hap',
  'meeteth',
  'heart—these',
  'strong-built',
  'rum',
  'availeth',
  'achaeams',
  'racked',
  'loud-resounding',
  'sepulchre',
  'fire-dogs',
  'succouring',
  'indignity',
  'marshallers',
  '271.1',
  'earth—who',
  'dolorous',
  'prayed—to',
  'clamour',
  'epicles',
  'hieth',
  'settest',
  'belloweth',
  'athwart',
  'wind-swept',
  'wavered',
  'aesepus',
  'awaiteth',
  'imbrius',
  'unrelentingly',
  'centre-piece',
  'lycomedes',
  'loveth',
  'shrieked',
  'drinking-bouts',
  'host.',
  'woes—',
  'cleareth',
  'levelling',
  'withal—',
  'aepytus',
  'watch-fires',
  'idas',
  'wands',
  'adjuring',
  'thersilochus',
  'eëriboea',
  'tasselled',
  'biddeth',
  'battlegear',
  'displeasing',
  'hyrtacus',
  'coif',
  'archelochus',
  'axius—',
  'sweepeth',
  'minisheth',
  'ptolemaeus',
  'selli',
  'babbler',
  'corpse—for',
  'rose-sweet',
  'rolleth',
  'skilless',
  'ensigns',
  'hips'

In [66]:
#Checkin how many unique terms there are

unique_terms = {term for values in tokenized_texts.values() for term in values['counts'].keys()} #List comprehension :O
print(f"There are {len(unique_terms)} unique terms across the texts, with")
for filename, values in tokenized_texts.items():
    unique_term_count = len(set(values['counts'].keys()))
    print(f"{unique_term_count} unique terms in {filename}")


There are 14669 unique terms across the texts, with
7809 unique terms in tlg0012.tlg001.perseus-eng3.txt
6604 unique terms in tlg0012.tlg002.perseus-eng4.txt
7750 unique terms in tlg0012.tlg001.perseus-eng4.txt
6511 unique terms in tlg0012.tlg002.perseus-eng3.txt
